<a href="https://colab.research.google.com/github/larregui/IST718-Project/blob/main/Final_Project_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Questions we would like to answer  
1. likelihood of UFO sighting given location, day of week, weather, etc - for this we will need to create a spatial heatmap
2. 

# Import necessary libraries

In [77]:
import pandas as pd #data frame operations
import numpy as np #arrays and math functions
from scipy.stats import uniform #for training and test splits
import statsmodels.api as sm  # statistical models (including regression)
import statsmodels.formula.api as smf  # R-like model specification
import matplotlib.pyplot as plt #2D plotting
import seaborn as sns #seaborn for plotting
from sklearn.linear_model import LinearRegression #not used
import os
import sys
import scipy.stats as stats
#plotly graphing
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
#For Spatial heatmaps
import folium
from folium import Choropleth
from folium.plugins import HeatMap

#Import Datasets

In [125]:
df=pd.read_csv('/content/ufo.csv')
# shape and data types of the data
print("What are the dimensions of the dataset?\n",df.shape)
print("What are the datatypes for each variable?\n",df.dtypes)
# compute descriptive statistics 
print("Descriptive Statistics\n",df.describe())
# % of missing.
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print("Are there any null values?\n",'{} - {}%'.format(col, round(pct_missing*100)))

What are the dimensions of the dataset?
 (80332, 11)
What are the datatypes for each variable?
 datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object
Descriptive Statistics
                 longitude 
count            80,332.00
mean                -86.77
std                  39.70
min                -176.66
25%                -112.07
50%                 -87.90
75%                 -78.75
max                 178.44
Are there any null values?
 datetime - 0.0%
Are there any null values?
 city - 0.0%
Are there any null values?
 state - 7.0%
Are there any null values?
 country - 12.0%
Are there any null values?
 shape - 2.0%
Are there any null values?
 duration (seconds) - 0.0%
Are the

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.



In [126]:
# read in Movie data
mov = pd.read_csv("alien_movies.csv")
# Drop unnecessary columns
mov = mov.drop([ 'Unnamed: 2', 'Unnamed: 3'], axis=1)
mov = mov.rename(columns={'Year':'year'})

In [127]:
# read in airport data
air = pd.read_csv("us-airports.csv")
air=air[['name','latitude_deg','longitude_deg']]
air = air.rename(columns={'latitude_deg':'latitude'})
air = air.rename(columns={'longitude_deg':'longitude'})
air

,name,latitude,longitude
0,Los Angeles International Airport,33.94,-118.41
1,Chicago O'Hare International Airport,41.98,-87.90
2,John F Kennedy International Airport,40.64,-73.78
3,Hartsfield Jackson Atlanta International Airport,33.64,-84.43
4,San Francisco International Airport,37.62,-122.38
...,...,...,...
23700,Florida Hospital Zephyrhills Heliport,28.26,-82.19
23701,Rotary Run Heliport,40.61,-81.42
23702,Andrew Othole Memorial Airport,35.06,-108.94
23703,Black Rock Heliport,35.08,-108.79


# Data Cleaning and Processing  

We decided to analize UFO reports in the US since the majority of the data was located in the US.  
The latitude, longitude, and sighting durations were converted to numeric values as they were not already numeric.

In [129]:
#print(df.country.unique())
#print(len(df.country),"Contries")
# The datase includes records from US, GB, CA, AU, and DE.
# Should we study one country? Yes
df = df[df.country == 'us']

df["latitude"] = pd.to_numeric(df["latitude"], errors="coerce")
df['longitude '] = pd.to_numeric(df["longitude "], errors="coerce")
df['duration (seconds)'] = pd.to_numeric(df["duration (seconds)"], errors="coerce")

#Only look at reports that have lasted about an hour
df = df[df['duration (seconds)'] <= 4000]

#dealing with null values
df.dropna(axis=0,inplace=True)

# Fix dates
df['datetime']=df['datetime'].astype(str)
df[['date','time']] = df['datetime'].str.split(expand=True)
df['datetime'] = (pd.to_datetime(df.pop('date'), format='%m/%d/%Y'))
df['year'] = pd.DatetimeIndex(df['datetime']).year

df['date posted']=df['date posted'].astype(str)
df[['date1']] = df['date posted'].str.split(expand=True)
df['date posted'] = (pd.to_datetime(df.pop('date1'), format='%m/%d/%Y'))  
                  

#reset index
df=df.reset_index(drop=True)

# save a copy of the dataframe
orig= df.copy(deep=True)

df.describe()


,duration (seconds),latitude,longitude,year
count,"61,148.00","61,148.00","61,148.00","61,148.00"
mean,477.53,38.32,-95.56,"2,004.27"
std,767.46,5.55,18.05,10.33
min,0.01,17.97,-176.66,"1,910.00"
25%,30.00,34.10,-114.46,"2,002.00"
50%,180.00,38.95,-89.92,"2,007.00"
75%,600.00,41.90,-80.89,"2,011.00"
max,"3,900.00",70.64,-65.83,"2,014.00"


In [130]:
df['shape'].nunique() #28 unique shapes
df['shape'].value_counts()

light        12885
triangle      6349
circle        5877
fireball      5078
unknown       4348
other         4260
sphere        4192
disk          3940
oval          2932
formation     1950
cigar         1599
changing      1436
rectangle     1042
flash         1024
cylinder       993
diamond        878
chevron        799
teardrop       579
egg            561
cone           232
cross          183
delta            5
round            2
flare            1
hexagon          1
changed          1
pyramid          1
Name: shape, dtype: int64

In [131]:
df['shape'] = df['shape'].replace({'Changed':'Changing',
                                       'Flare':'Fireball'}, regex=True)

In [132]:
def shape_category_counter(comments, shape_categories):
    comments = comments.lower().split()
    cat_count = dict(zip(list(shape_categories), [0] * len(shape_categories)))
    for word in comments:
        if word in shape_categories:
            cat_count[word] += 1
    cat_count = {k[0].upper()+k[1:]:v for k, v in cat_count.items() if v}
    return list(cat_count.keys())

In [133]:
shape_categories = [i.lower() for i in df['shape'].value_counts().index if i not in ['Unknown','Other']]
df['shapeCat'] = df['comments'].apply(lambda x: shape_category_counter(x, shape_categories))

# EDA

#Merge Datasets

In [134]:
# Merge all dataframes
df1 = df.merge(mov, on='year', how='left')
#Something wrong happened with the shapeCat
#df['shapeCat']
df1

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,time,year,shapeCat,Title
0,1949-10-10,san marcos,tx,us,cylinder,"2,700.00",45 minutes,This event took place in early fall around 194...,2004-04-27,29.88,-97.94,20:30,1949,[],NaN
1,1956-10-10,edna,tx,us,circle,20.00,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.98,-96.65,21:00,1956,[],Earth vs. the Flying Saucers
2,1956-10-10,edna,tx,us,circle,20.00,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.98,-96.65,21:00,1956,[],Fire Maidens from Outer Space
3,1956-10-10,edna,tx,us,circle,20.00,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.98,-96.65,21:00,1956,[],Forbidden Planet
4,1956-10-10,edna,tx,us,circle,20.00,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.98,-96.65,21:00,1956,[],Invasion of the Body Snatchers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748268,2013-09-09,edmond,ok,us,cigar,"1,020.00",17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,2013-09-30,35.65,-97.48,23:00,2013,[],Tropico
748269,2013-09-09,edmond,ok,us,cigar,"1,020.00",17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,2013-09-30,35.65,-97.48,23:00,2013,[],Unaware
748270,2013-09-09,edmond,ok,us,cigar,"1,020.00",17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,2013-09-30,35.65,-97.48,23:00,2013,[],Under the Skin
748271,2013-09-09,edmond,ok,us,cigar,"1,020.00",17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,2013-09-30,35.65,-97.48,23:00,2013,[],V/H/S/2


# Time Series

# Text Mining

In [135]:
#text=pd.DataFrame(df['comments'])
#re-index for peace of mind
df = df.reset_index(drop = True)
df['comments'] = df['comments'].str.replace("[^a-zA-Z]", " ")


In [136]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
# SCRUB - TOKEN

# tokenization 
tokenized_doc = df['comments'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['comments'] = detokenized_doc

In [138]:
tokenized_doc=tokenized_doc.astype(str)

#Function to convert List of strings to a string with a separator
def converttostr(input_seq, seperator):
   # Join all the strings in list
   final_str = seperator.join(input_seq)
   return final_str

# List of words separated with a comma
seperator = ', '
strlist=converttostr(tokenized_doc, seperator)

from nltk.tokenize import word_tokenize# Passing the string text into word tokenize for breaking the sentences
token = word_tokenize(strlist)


In [139]:
# finding the frequency distinct in the tokens
# Importing FreqDist library from nltk and passing token into FreqDist
from nltk.probability import FreqDist
fdist = FreqDist(token)
#fdist

In [148]:
# To find the frequency of top 20 words
fdist1 = fdist.most_common(20)
fdist1

[("'", 585846),
 (',', 585807),
 ('[', 61148),
 (']', 61148),
 ("'sky", 12482),
 ("'lights", 12479),
 ("'light", 12351),
 ("'I", 10857),
 ("'object", 10162),
 ("'moving", 6861),
 ("'bright", 6546),
 ("'shaped", 5201),
 ("'white", 5182),
 ("'orange", 5057),
 ("'red", 4804),
 ("'saw", 4424),
 ("'seen", 4091),
 ("'like", 3991),
 ("'craft", 3683),
 ("'flying", 3563)]

# Spatial Heatmaps

In [112]:
def generateMap(self, settings):
        """Generates the heatmap.
        
        Arguments:
            settings {dict} -- The settings for the heatmap.
        
        Returns:
            Map -- The Heatmap.
        """
        tiles = settings["tiles"]
        zoom_start = settings["zoom_start"]
        radius = settings["radius"]
        blur = settings["blur"]
        min_opacity = settings["min_opacity"]
        max_zoom = settings["max_zoom"]
        
        meanLong = statistics.mean(df['longitude '])
        meanLat = statistics.mean(df['latitude'])
        
        map_data = [(latitude, longitude )
                    for latitude, longitude in df]

        # Generate map
        m = folium.Map(location=[meanLat, meanLong],
                       zoom_start=zoom_start,
                       tiles=tiles)

        # Generate heat map
        heatmap = HeatMap(map_data,
                          #max_val=self.max_magnitude,
                          min_opacity=min_opacity,
                          radius=radius,
                          blur=blur,
                          max_zoom=max_zoom)

        m.add_child(heatmap)
        return m 


In [146]:
# calculating mean longitude and latitude values
import statistics
meanLong = statistics.mean(df['longitude '])
meanLat = statistics.mean(df['latitude'])
map_data = (df['latitude'], df['longitude '])
# create base map object using Map()
m_1 = folium.Map(location=[40,-97.941111], zoom_start = 14.5)
# create heatmap layer
HeatMap(map_data, radius=10).add_to(m_1)
# Show the map
m_1

#True or Not

In [103]:
#If there was a movie the same year then it is not credible
df1.loc[df1['Title'] != 'NaN', 'Credible'] = 'Fake'  


In [173]:
import datetime
data = df[['shape','latitude','longitude ','datetime','comments']]
data['datetime'] = data['datetime'].apply(lambda x: pd.Timestamp(x).date())
data = data[(data['datetime'] <= datetime.date(2020, 1, 1))]
    
cols = ['shape', 'latitude', 'longitude ', 'datetime']
data['reports'] = data.groupby(cols).transform('count')
data.loc[data['reports'] >=2, 'Credible'] = 'YES' 
data.loc[data['reports'] <2, 'Credible'] = 'NO' 
#df['Credible?'] = df['reports'].apply(lambda x: 'Yes' if x >= 4 else 'No')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [176]:
data['Credible'].unique

<bound method Series.unique of 0        NO
1        NO
2        NO
3        NO
4        NO
         ..
61143    NO
61144    NO
61145    NO
61146    NO
61147    NO
Name: Credible, Length: 61148, dtype: object>

In [169]:
import sklearn
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import TfidfVectorizer

#Define features (summaries) and labels (credibilities)
summaries = data['comments']
credibilities = data['Credible']

#Vectorize features, aka text summaries.
vectorizer = TfidfVectorizer(ngram_range=(1,3), min_df=5)
vectors = vectorizer.fit_transform(summaries)

f_train, f_test, l_train, l_test = train_test_split(vectors, credibilities, test_size = 0.3)
svm = LinearSVC(C=0.7, class_weight='balanced')
clf = CalibratedClassifierCV(svm)
clf.fit(f_train, l_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=0.7, class_weight='balanced',
                                                dual=True, fit_intercept=True,
                                                intercept_scaling=1,
                                                loss='squared_hinge',
                                                max_iter=1000,
                                                multi_class='ovr', penalty='l2',
                                                random_state=None, tol=0.0001,
                                                verbose=0),
                       cv=None, method='sigmoid')

In [170]:
pred_train = clf.predict(f_train)
print(classification_report(l_train, pred_train))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

          NO       0.97      1.00      0.98     41308
         YES       0.00      0.00      0.00      1495

    accuracy                           0.97     42803
   macro avg       0.48      0.50      0.49     42803
weighted avg       0.93      0.97      0.95     42803

